In [26]:
import warnings

warnings.filterwarnings("ignore")

import os
from torch.utils import data
from dataset import*
from config import settings
from train_test import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import time

In [27]:
transformers = {
        'year': None, # Normalize
        'weekday': None,
        'time': None, # Standardlize
        'weather': None
    }
# print('loadind data...')
train_df, transformers = load_data('train', total_sample=10000000, random_sample=settings.totalN, scaling_transformers=transformers)
# print(f'\nAfter feature engineering: {train_df.columns},\n total len: {len(train_df.columns)}\n')
test_df, transformers = load_data('test', random_sample=settings.totalN, scaling_transformers=transformers)
train_data_df = train_df.copy()
test_data_df = test_df.copy()
#train_data_df.drop('fare_amount', axis=1, inplace=True)
train_data_df.drop('net_fare', axis=1, inplace=True)
#train_data_df.drop('total_fixed_fees', axis=1, inplace=True)
#test_data_df.drop('total_fixed_fees', axis=1, inplace=True)
test_data_df.drop('key', axis=1, inplace=True)

loaded csv file shape: (10000000, 8)
setting time info...
basic time information costing time: 20.306
advanced time information costing time: 14.628
setting geo info...
geo information costing time: 3.503
counting net fare...
counting fixed fee costing time: 0.065
data flitering time: 11.448
loaded test csv file shape: (9914, 7)
setting time info...
basic time information costing time: 0.014
advanced time information costing time: 0.012
setting geo info...
geo information costing time: 0.011
counting fixed fee...
counting fixed fee costing time: 0.000
data flitering time: 0.001


In [30]:
X_train = train_data_df.drop('fare_amount', axis=1)
y_train = train_data_df['fare_amount']

In [35]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

xgb_reg = xgb.XGBRegressor()
# 設定要調整的超參數範圍
params = {
    'learning_rate': [0.01, 0.1, 0.5, 1],
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.5, 0.7, 1],
    'colsample_bytree': [0.5, 0.7, 1],
}
# 初始化GridSearchCV
grid_search = GridSearchCV(xgb_reg, params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

In [36]:
# 訓練模型
print('training model...')
start_time = time.time()
# 擬合數據
grid_search.fit(X_train, y_train)
# 找到最佳參數
best_params = grid_search.best_params_
print(f"最佳參數: {best_params}")
# 使用最佳參數訓練模型
best_xgb_reg = xgb.XGBRegressor(**best_params)
best_xgb_reg.fit(X_train, y_train)

training model...


KeyboardInterrupt: 

In [ ]:
def format_time(seconds):
    minutes = seconds // 60
    hours = minutes // 60

    if hours > 0:
        return f"{hours} hr {minutes % 60} min {seconds % 60} sec"
    elif minutes > 0:
        return f"{minutes} min {seconds % 60} sec"
    else:
        return f"{seconds} sec"

In [ ]:
end_time = time.time()
elapsed_time = end_time - start_time
print(f'total training time: {format_time(int(elapsed_time))}')
print('predict result...')
# 用訓練好的模型來預測測試資料集
predictions = best_xgb_reg.predict(test_data_df)
# 將預測結果轉換為DataFrame
total_fixed_fees = test_df['total_fixed_fees'].values
predictions_df = pd.DataFrame(predictions+total_fixed_fees, columns=['fare_amount'])
# 從測試數據集中提取'key'欄位
keys_df = test_df['key']
# 將'key'與預測結果組合成一個新的DataFrame
result_df = pd.concat([keys_df.reset_index(drop=True), predictions_df], axis=1)
result_df.to_csv('result.csv', index=False)

training model...
total training time: 21 min 44 sec
predict result...
